Projeto 2 de Ciência dos Dados

In [2]:
#Carregando as bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\rafae\Desktop\cdados_p2\cdados_projeto2


In [54]:
#Lendo o dataset
data = pd.read_csv("C:/Users/rafae/Desktop/cdados_p2/cdados_projeto2/matches_info.csv")

#Apagando colunas desnecessárias
del data["Unnamed: 0"]
del data["date_match"]
del data["event_name"]

#Criando um novo dataframe somente com as partidas, mapas e times, excluindo as estatisticas de cada jogador, pois
#existem alguns jogadores que atualmente não estão mais participando do cenário competitivo do jogo.
data_novo = data.iloc[:, 0:23]

#Esta função faz com que todas as colunas sejam expostas pelo DataFrame.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

#Função que conta a quantidade de aparições dos mapas, retornando um dicionário com cada um deles e suas frequencias
def contagem(lista):
    dicionario_contagem = {}
    for i in lista:
        dicionario_contagem[i] = lista.count(i)
        
    return dicionario_contagem

In [69]:
#Criando uma lista que contenha o nome de todos os times
lista_times1 = list(data_novo.team1)
lista_times2 = list(data_novo.team2)
lista_times = list(lista_times1+lista_times2)

#Criando uma lista que contenha o nome de todos os mapas
lista_mapas = []
lista_frq_mapas = []
lista_crua = list(data_novo.map1_played) + list(data_novo.map2_played) + list(data_novo.map3_played)

for i in lista_crua:
    if i in lista_mapas or i == "NotPlayed":
        pass
    else:
        lista_mapas.append(i)
        
#Criando uma lista para medir a frequencia de jogos nos mapas
for i in lista_crua:
    if i == "NotPlayed":
        pass
    else:
        lista_frq_mapas.append(i)

In [95]:
#Criando um dicionário que mostra a frequencia que cada mapa é jogado
soma_mapas = sum(contagem(lista_frq_mapas).values())
dicionario_frequencias_mapas = {}
for k, v in contagem(lista_frq_mapas).items():
    frq = v*(100) / soma_mapas
    dicionario_frequencias_mapas[k] = frq

#Criando um dicionário que mostra a frequencia dos times em relação as partidas jogadas
soma_times = sum(contagem(lista_times).values())
dicionario_frequencias_times = {}
for k, v in contagem(lista_times).items():
    frq = v*(100) / soma_times
    dicionario_frequencias_times[k] = frq

#Código que cria um dicionário que mostra a porcentagem de vitória dos times a partir da soma de todas as partidas computadas.
lista_mapa2 = []
lista_mapa3 = []

for i in data_novo.team_winner_map2:
    if i == "NotPlayed":
        pass
    else:
        lista_mapa2.append(i)

for i in data_novo.team_winner_map3:
    if i == "NotPlayed":
        pass
    else:
        lista_mapa3.append(i)

vencedores = list(data_novo.team_winner_map1) + lista_mapa2 + lista_mapa3
soma_vencedores = sum(contagem(vencedores).values())
dicionario_frequencias_vencedores = {}
for k, v in contagem(vencedores).items():
    frq = v*(100) / soma_vencedores
    dicionario_frequencias_vencedores[k] = frq
    
#Organiza o dicionário em ordem decrescente, facilitando sua visualização
dicionario_vencedores_sorted = dict(sorted(dicionario_frequencias_vencedores.items(), key=lambda x: x[1], reverse=True))